In [2]:
import pandas as pd

file_path = 'C:/Users/moura/PAD_JOB/data_extracted/New-dataset-SVCpredictions.csv'
dataset = pd.read_csv(file_path)


dataset.head()

,title,text,sentiment,stars,likesCount,reviewerPhotoUrl,publishedAtDate,name,predictions
0,BOB'S,Shake e Sunday aguados. Horrível. Não recomendo.,negative,1,1,https://lh3.googleusercontent.com/a-/AD_cMMRso...,2021-04-03 20:10:01.702000+00:00,Sílvio Brito,negative
1,BOB'S,Credo!,negative,1,0,https://lh3.googleusercontent.com/a-/AD_cMMS7i...,2019-05-25 21:11:05.280000+00:00,Tiago de Bem,negative
2,BOB'S,"Desorganizado, pedido veio errado",negative,1,0,https://lh3.googleusercontent.com/a-/AD_cMMQP2...,2020-02-15 23:29:17.166000+00:00,paulo aecio alves,negative
3,BOB'S,Atendimento péssimo,negative,1,0,https://lh3.googleusercontent.com/a-/AD_cMMSwu...,2018-10-18 10:25:35.416000+00:00,Eryka Ferreira Menezes de Paula,negative
4,BOB'S,"Todos atendentes de mal humor e sem educação, ...",negative,1,2,https://lh3.googleusercontent.com/a-/AD_cMMS_z...,2022-01-15 18:37:15.511000+00:00,João Benuto,negative


In [3]:
dataset.iloc[2].at['text']

'Desorganizado, pedido veio errado'

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89212 entries, 0 to 89211
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             89212 non-null  object
 1   text              89212 non-null  object
 2   sentiment         89212 non-null  object
 3   stars             89212 non-null  int64 
 4   likesCount        89212 non-null  int64 
 5   reviewerPhotoUrl  89212 non-null  object
 6   publishedAtDate   89212 non-null  object
 7   name              89212 non-null  object
 8   predictions       89212 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.1+ MB


In [5]:
grouped = dataset.groupby('title').agg(
    #contando quantas vezes title ocorreu
    count_titulos=('title', 'count'),
    #soma das estrelas
    sum_estrelas=('stars', 'sum')
).reset_index()

grouped = dataset.groupby('title').agg(
    count_titulos=('title', 'count'),
    sum_estrelas=('stars', 'sum')
).reset_index()

grouped['media_estrelas'] = grouped['sum_estrelas'] / grouped['count_titulos']


frequent_titles = grouped[grouped['count_titulos'] >= 10]
top_titles = frequent_titles.sort_values('media_estrelas', ascending=False).head(10)
bottom_titles = frequent_titles.sort_values('media_estrelas', ascending=True).head(10)


display(grouped)

,title,count_titulos,sum_estrelas,media_estrelas
0,20 Buscar Bar e Restaurante,184,677,3.679348
1,"597 Burger (Hambúrguer, Batata frita, Cerveja)...",407,1807,4.439803
2,6 Pira Açai,10,49,4.900000
3,A GRANDE FAMÍLIA RESTAURANTE,22,101,4.590909
4,ABSOLUTO BAR & RESTAURANTE,253,1224,4.837945
...,...,...,...,...
843,brasa burg's,7,30,4.285714
844,buchecha burg,1,5,5.000000
845,restaurante Caseiro casa de vó,7,35,5.000000
846,tioBák - Passeio das Àguas,106,411,3.877358


In [6]:
#estabelecimentos com maior número de comentários
top_coments = frequent_titles.sort_values('count_titulos', ascending=False).head(10)

print(top_coments)

                                              title  count_titulos  \
281                       Júnior Cozinha Brasileira           5154   
385                       Outback Passeio das Águas           2618   
350                                      Mau Nenhum           2004   
114                             Carne de Sol do Edu           1998   
298                         Labodeguita Restaurante           1748   
619                             Restaurante Popular           1243   
759                 Simbora Restaurante 24h - Urias           1226   
368  Mourão II Chopp e Restaurante - Setor Sudoeste           1151   
310                    Libanu's Restaurante Goiânia           1083   
7                      Abruzzo Restaurante Italiano           1079   

     sum_estrelas  media_estrelas  
281         24347        4.723904  
385         12039        4.598549  
350          8858        4.420160  
114          9060        4.534535  
298          7829        4.478833  
619          

In [7]:

display(top_titles)

,title,count_titulos,sum_estrelas,media_estrelas
154,Cozinha da Simone restaurante,10,50,5.000000
826,Wil Burguer,13,65,5.000000
794,Tom Jr Burguer,21,105,5.000000
8,Acaraje da Baiana,23,115,5.000000
261,Hamburguer na Brasa e Jantinhas Toma ma da ka,18,90,5.000000
87,Bão Demais Pamonharia Goiana,14,70,5.000000
587,Restaurante House Food Gastrobar,11,55,5.000000
24,Açaiteria da Jana,16,80,5.000000
13,Alô Massa Restaurante de Macarrão e Massas del...,41,204,4.975610
209,Espetinho do GuGu,22,109,4.954545


Análise de Sentimentos por Comércio


In [8]:

display(bottom_titles)

,title,count_titulos,sum_estrelas,media_estrelas
75,Bom Prato,17,35,2.058824
29,BAPI - Shopping Flamboyant,34,75,2.205882
287,Kid abelha - Noroeste,13,30,2.307692
458,Primus pizzaria e Restaurante,61,158,2.590164
247,Giraffas,53,142,2.679245
18,Armazémm 73,107,306,2.859813
315,Lug's Passeio Das Águas Shopping - Goiânia,43,123,2.860465
461,QG Jeitinho Caseiro,29,86,2.965517
361,"Mimy Açaí - Vale dos Sonhos, Nativo da Natividade",13,42,3.230769
360,Metrô food pq.santa rita,35,114,3.257143


In [17]:
title_especifico = grouped[grouped['title'] == "Pizza z"]
print(title_especifico[['title', 'media_estrelas']])


Empty DataFrame
Columns: [title, media_estrelas]
Index: []
